In [37]:
import numpy as np
import os
import statistics as st
import matplotlib.pyplot as plt
from scipy.optimize import least_squares
from tqdm import tqdm


all_gt=np.array([]).reshape(0,2)

frame_max=0

for i in range(1,2):
    gt=np.load(r'D:\Kinect Tools\Nahian\Code Backup Mar 29\All_code_f\Neo_offline_track\GT data\Feb 2_per_frame\ video_gtpf_'+str(i)+'.npy')
    gt=np.load(r'D:\Kinect Tools\Nahian\Code Backup Mar 29\All_code_f\Neo_offline_track\GT data\Jan 19_per_frame\ video_gtpf_'+str(i)+'.npy')
    print(gt[-1,0])
    frame_max=max(frame_max,gt[-1,0])
    if i>1:
        gt[:,0]=gt[:,0]+frame_max
    frame_max=gt[-1,0]
    print("   Frame Max frame for video= ",str(i)," = ",frame_max)
    print("Max frame for video= ",str(i)," = ",np.max(gt[:,0]))
    all_gt=np.concatenate((all_gt,gt),axis=0)
    print("  and last frame = ",all_gt[-1,0])
print(all_gt.shape)
print("   Frame Max frame for all video = ",frame_max)
print("Max frame for all video = ",np.max(all_gt[:,0]))
    
a=np.load(r'D:\Kinect Tools\Nahian\Code Backup Mar 29\All_code_f\Neo_offline_track\output_data_processed\Feb 2_v6\video_1.npy')
#print(a.shape)
#print(a[1000,:])
#print(np.max(a[:,0]))

#load all video data
frame_max=0
all_data=np.array([]).reshape(-1,a.shape[1])
for i in range(1,2):
    a=np.load(r'D:\Kinect Tools\Nahian\Code Backup Mar 29\All_code_f\Neo_offline_track\output_data_processed\Feb 2_v6\video_'+str(i)+'.npy')
    a=np.load(r'D:\Kinect Tools\Nahian\Code Backup Mar 29\All_code_f\Neo_offline_track\output_data_processed\Jan 19_2\video_'+str(i)+'.npy')
    print(a.shape)
    frame_max=max(frame_max,np.max(a[:,0]))
    if i>1:
        a[:,0]=a[:,0]+frame_max
    frame_max=np.max(a[:,0])
    print("   Frame Max frame for video= ",str(i)," = ",frame_max)
    print("Max frame for video= ",str(i)," = ",np.max(a[:,0]))
    all_data=np.vstack((all_data,a))
    print(" and Last frame= ",all_data[-1,0])
print(all_data.shape)

print(all_data[-1,0])


#sample the data where activity class is 6

#Delete 6
b=np.where(all_gt[:,1]==6)[0]
print("Having 6: ",b.shape)
frames_to_delete=all_gt[b,0]
print(frames_to_delete.shape)
all_gt_sampled=np.delete(all_gt,b,0)
print(all_gt_sampled.shape)
b_ver=np.where(all_gt_sampled[:,1]==6)[0]
print(b_ver.shape)



#delete the frames from all_data
b=[indices for indices, x in enumerate(all_data[:,0]) if x in frames_to_delete]
print(len(b))
all_data_sampled=np.delete(all_data,b,0)


#Delete 5
b=np.where(all_gt_sampled[:,1]==5)[0]
print("Having 5: ",b.shape)
frames_to_delete=all_gt_sampled[b,0]
all_gt_sampled=np.delete(all_gt_sampled,b,0)
print("All gt sampled= ",all_gt_sampled.shape)
b_ver=np.where(all_gt_sampled[:,1]==5)[0]
print(b_ver.shape)

print(frames_to_delete.shape)

#delete the frames from all_data
b=[indices for indices, x in enumerate(all_data_sampled[:,0]) if x in frames_to_delete]
print(len(b))
all_data_sampled=np.delete(all_data_sampled,b,0)


#Delete 4
b=np.where(all_gt_sampled[:,1]==4)[0]
print("Having 4: ",b.shape)
print("All gt sampled= ",all_gt_sampled.shape)
frames_to_delete=all_gt_sampled[b,0]
all_gt_sampled=np.delete(all_gt_sampled,b,0)
print("All gt sampled= ",all_gt_sampled.shape)
b_ver=np.where(all_gt_sampled[:,1]==4)[0]
print(b_ver.shape)

print(frames_to_delete.shape)

#delete the frames from all_data
b=[indices for indices, x in enumerate(all_data_sampled[:,0]) if x in frames_to_delete]
print(len(b))
all_data_sampled=np.delete(all_data_sampled,b,0)



unique_frames=np.unique(all_data_sampled[:,0])
print(unique_frames.shape)
print(all_gt_sampled.shape)

#One hot encoding

all_gt_sampled_oh=np.zeros((all_gt_sampled.shape[0],4))

for i in range(all_gt_sampled.shape[0]):
    all_gt_sampled_oh[i,int(all_gt_sampled[i,1])-1]=1



bolod=0


def close_point(heads,unit_vects):
    #global bolod
    #bolod=bolod+1
    heads=np.transpose(heads)

    Ttr=unit_vects
    #unit_vects=-1*unit_vects
    unit_vects=np.transpose(unit_vects)
    T=np.transpose(Ttr)

    I=np.identity(3)
    C_prime=np.zeros(heads.shape)
    #sum_v=np.zeros((3,3,heads.shape[1]))
    #sum_c=0
    for i in range(heads.shape[1]):
        #C_prime[:,i]=heads[:,i]-np.dot(heads[:,i],unit_vects[:,i])*unit_vects[:,i]
        C_prime[:,i]=heads[:,i]-unit_vects[:,i]*np.matmul(np.transpose(heads[:,i]),unit_vects[:,i])
        #sum_v=sum_v+np.matmul(unit_vects[:,i],np.transpose(unit_vects[:,i]))
        #sum_v[:,:,i]=np.outer(unit_vects[:,i],unit_vects[:,i])

        #print("Trouble shoot",np.dot(C_prime[:,i],unit_vects[:,i]))
    sum_c=np.sum(C_prime,axis=1)
    #sum_v=np.nan_to_num(sum_v/(heads.shape[1]))
    #sum_v=np.nan_to_num(np.sum(sum_v,axis=2)/(heads.shape[1]))
    
    sum_c=sum_c/heads.shape[1]
    #sum_v_inv=np.linalg.inv((I-sum_v))
    
    V=np.nan_to_num(np.matmul(T,Ttr))

    #print("V= ",V)
    #print("Det of V= ",np.linalg.det(V))

    sum_v_inv=np.linalg.pinv(I-V/heads.shape[1])
    #sum_v_inv=np.linalg.pinv(I-sum_v)#/heads.shape[1])

    #print("sum_v_inv= ",sum_v_inv)
    """
    w,_=np.linalg.eig(V)
    trc=np.trace(V)
    if bolod%100==0:
        print("Eigen values are ",w)
        print("Trace is ",trc)
        print("Determinant is ",np.linalg.det(V))
        #print("Sum of eigen values is ",np.sum(w))
        print("HUDA trace",np.trace(sum_v))
        print("Huda det ",np.linalg.det(sum_v))
        print("Huda eigen values ",np.linalg.eig(sum_v)[0])
    """
    #uniq=np.unique(w)
    #if len(uniq)!=3 or np.linalg.det(sum_v_inv)==0:
    #    print("Eigen values are not unique")
    #    print("Eigen values are ",w)
    X=np.matmul(sum_v_inv,sum_c)
    X=np.nan_to_num(np.transpose(X))
    return X



def objective(point, *points):
    distances = []
    for p in points:
        distances.append(np.linalg.norm(p - point))
    return distances




# sample input points
points = np.array([[1,1,1], [-1,1,1], [1,1,1],[-1,1,1],[1,1,1]])

#points = np.array([[1,1,1]])
print(points.shape)

# initial guess for the solution point
x0 = np.array([0,0,1])

# perform least squares optimization
result = least_squares(objective, x0, args=tuple(points))


print(result.x)


#Feature extraction

all_features=np.zeros((all_gt_sampled_oh.shape[0],6))

i=0
for frames in tqdm(unique_frames):
#for frames in unique_frames:
    b=np.where(all_data_sampled[:,0]==frames)[0]
    current_frame=all_data_sampled[b,:]
    poses=(current_frame[:,3]>1300).astype(int).reshape(current_frame.shape[0],1) 
    all_features[i,3]=sum(poses)#standing
    all_features[i,4]=current_frame.shape[0]-sum(poses)#sitting

        #Default
    sampled_current_frame=current_frame.copy()
    sampled_points=sampled_current_frame[:,11:14]/(np.linalg.norm(sampled_current_frame[:,11:14],axis=1).reshape(-1,1))
    sampled_heads=sampled_current_frame[:,1:4]

    #x0 = np.array([1,1,1])
    #result = least_squares(objective, x0, args=tuple(sampled_points))
    #gaze_point=result.x/np.linalg.norm(result.x)
    gaze_point=close_point(sampled_heads,sampled_points)

    all_features[i,0:3]=gaze_point


    if sum(poses)>0 and sum(poses)<current_frame.shape[0]: #Some standing, some sitting
        y_values=np.zeros((current_frame.shape[0],1))
        x_values=np.zeros((current_frame.shape[0],1))
        z_values=np.zeros((current_frame.shape[0],1))
        flag_presentation=0
        
        
        if sum(poses)<=0.5*current_frame.shape[0]: #Most people sitting
            for j in range(current_frame.shape[0]):
                y_values[j]=current_frame[j,2]
                x_values[j]=current_frame[j,1]
                z_values[j]=current_frame[j,3]
            
            alu_gaze=[]

            for j in range(current_frame.shape[0]):
                if (y_values[j]>9600 and x_values[j]>1850 and x_values[j]<4200 and z_values[j]>1300) or (y_values[j]>9200 and x_values[j]>2400 and x_values[j]<4200 and z_values[j]>1300) : # Presenter is standing, most people sitting
                    flag_presentation=1
                    all_features[i,5]=1
                    other_ids=np.delete(np.arange(current_frame.shape[0]),j)
                    #sampled_current_frame=current_frame[other_ids,:]
                    sampled_current_frame=current_frame.copy()
                    sampled_points=sampled_current_frame[:,11:14]/(np.linalg.norm(sampled_current_frame[:,11:14],axis=1).reshape(-1,1))
                    sampled_heads=sampled_current_frame[:,1:4]

                    #x0 = np.array([1,1,1])
                    #result = least_squares(objective, x0, args=tuple(sampled_points))
                    #gaze_point=result.x/np.linalg.norm(result.x)
                    gaze_point=close_point(sampled_heads,sampled_points)

                    all_features[i,0:3]=gaze_point
                    break
    else:
        all_features[i,5]=0
        sampled_points=current_frame[:,11:14]/(np.linalg.norm(current_frame[:,11:14],axis=1).reshape(-1,1))
        sampled_heads=current_frame[:,1:4]
        
        #x0 = np.array([1,1,1])
        #result = least_squares(objective, x0, args=tuple(sampled_points))
        #gaze_point=result.x/np.linalg.norm(result.x)
        gaze_point=close_point(sampled_heads,sampled_points)
        all_features[i,0:3]=gaze_point
    i=i+1



    #print(all_data_sampled[b,1:])
    


20992.0
   Frame Max frame for video=  1  =  20992.0
Max frame for video=  1  =  20992.0
  and last frame =  20992.0
(20992, 2)
   Frame Max frame for all video =  20992.0
Max frame for all video =  20992.0
(76099, 17)
   Frame Max frame for video=  1  =  20992.0
Max frame for video=  1  =  20992.0
 and Last frame=  20992.0
(76099, 17)
20992.0
Having 6:  (0,)
(0,)
(20992, 2)
(0,)
0
Having 5:  (0,)
All gt sampled=  (20992, 2)
(0,)
(0,)
0
Having 4:  (1590,)
All gt sampled=  (20992, 2)
All gt sampled=  (19402, 2)
(0,)
(1590,)
2106
(19402,)
(19402, 2)
(5, 3)
[0.19996011 0.99999973 0.99999999]


  0%|          | 0/19402 [00:00<?, ?it/s]C:\Users\Nahian\AppData\Local\Temp/ipykernel_23224/3552117195.py:225: RuntimeWarning: invalid value encountered in true_divide
  sampled_points=sampled_current_frame[:,11:14]/(np.linalg.norm(sampled_current_frame[:,11:14],axis=1).reshape(-1,1))
C:\Users\Nahian\AppData\Local\Temp/ipykernel_23224/3552117195.py:270: RuntimeWarning: invalid value encountered in true_divide
  sampled_points=current_frame[:,11:14]/(np.linalg.norm(current_frame[:,11:14],axis=1).reshape(-1,1))
100%|██████████| 19402/19402 [00:08<00:00, 2212.67it/s]


In [38]:
print(all_data_sampled.shape)
print(all_data_sampled[-1,0])
print(np.min(all_data_sampled[:,0]))
print(np.max(unique_frames))
print(unique_frames.shape)
print(all_features.shape)

(73993, 17)
20992.0
421.0
20992.0
(19402,)
(19402, 6)


In [39]:
gaze_features=np.array([]).reshape(0,3)
gaze_features=np.concatenate((gaze_features,all_features[:,0:3]),axis=0)
gaze_features=np.hstack((unique_frames.reshape(-1,1),gaze_features))
print(gaze_features.shape)

(19402, 4)


In [40]:
print(all_data_sampled[0,0:3])
print(gaze_features[-100,0:4])

[421.   0.   0.]
[20893.          2382.55684739  7590.98388418   449.77432566]


In [41]:
np.save('gaze_features_jan4.npy',gaze_features)
np.save('all_data_sampled_jan4.npy',all_data_sampled)

In [42]:
for i in range(gaze_features.shape[0]):
    print(gaze_features[i,0:4])

[421.   0.   0.   0.]
[422.   0.   0.   0.]
[423.   0.   0.   0.]
[424.   0.   0.   0.]
[425.   0.   0.   0.]
[426.   0.   0.   0.]
[427.   0.   0.   0.]
[428.   0.   0.   0.]
[429.   0.   0.   0.]
[430.   0.   0.   0.]
[431.   0.   0.   0.]
[432.   0.   0.   0.]
[433.   0.   0.   0.]
[434.   0.   0.   0.]
[435.   0.   0.   0.]
[436.   0.   0.   0.]
[437.   0.   0.   0.]
[438.   0.   0.   0.]
[439.   0.   0.   0.]
[440.   0.   0.   0.]
[441.   0.   0.   0.]
[442.   0.   0.   0.]
[443.   0.   0.   0.]
[444.   0.   0.   0.]
[445.   0.   0.   0.]
[446.   0.   0.   0.]
[447.   0.   0.   0.]
[448.   0.   0.   0.]
[449.   0.   0.   0.]
[450.   0.   0.   0.]
[451.   0.   0.   0.]
[452.   0.   0.   0.]
[ 453.         2206.40729465 9629.2475208   395.06980359]
[ 454.         2206.40729465 9629.2475208   395.06980359]
[ 455.         2206.40729465 9629.2475208   395.06980359]
[ 456.         2206.40729465 9629.2475208   395.06980359]
[ 457.         2206.40729465 9629.2475208   395.06980359]
[ 458.

In [212]:
print(np.divide(gaze_features[100:107,1:4],np.linalg.norm(gaze_features[100:107,1:4],axis=1).reshape(-1,1)))
print(np.linalg.norm(gaze_features[100:107,1:4],axis=1).reshape(-1,1))

[[-0.64057032  0.41779366 -0.64429662]
 [-0.63557108  0.42142306 -0.64687866]
 [-0.58313397  0.37254043 -0.72191995]
 [-0.44796454  0.26828076 -0.85285005]
 [-0.35795217  0.23136222 -0.90462244]
 [-0.41647211  0.27047736 -0.86798213]
 [-0.37865815  0.24090993 -0.89363327]]
[[7738.90142251]
 [7762.12105169]
 [7534.26580598]
 [7099.54725461]
 [7020.92033512]
 [7167.60620793]
 [7071.27928215]]


In [156]:
w=np.array([[300,20,200]])
print(w/10)

[[30.  2. 20.]]
